# SNAC API grab remastered

In honor of trying to get the "People" section on the Bond Papers site up and running, let's try to improve, well, everything about my old SNAC datagrab from summer '23. Need to postpone this until I can get ahold of my old laptop, though, I think. Can't seem to recover it from elsewhere.

In [5]:
import numpy as np
np.__version__

'2.3.0'

In [7]:
import time
from datetime import datetime
import pandas as pd
import requests
import json

In [9]:
query = """
{
    "command": "read",
    "constellationid": 83834110
}
"""
url = 'https://api.snaccooperative.org/'

In [15]:
requests.put??

Signature: requests.put(url, data=None, **kwargs)
Source:   
def put(url, data=None, **kwargs):
    r"""Sends a PUT request.

    :param url: URL for the new :class:`Request` object.
    :param data: (optional) Dictionary, list of tuples, bytes, or file-like
        object to send in the body of the :class:`Request`.
    :param json: (optional) A JSON serializable Python object to send in the body of the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response
    """

    return request("put", url, data=data, **kwargs)
File:      c:\users\charl\anaconda3\lib\site-packages\requests\api.py
Type:      function

In [11]:
response = requests.put(url, query)
print(response)
print(response.text)

<Response [200]>
{
    "result": "success",
    "maybesame_count": 0,
    "constellation": {
        "dataType": "Constellation",
        "status": "editable",
        "ark": "http:\/\/n2t.net\/ark:\/99166\/w6jv0dh3",
        "entityType": {
            "id": "700",
            "term": "person",
            "uri": "http:\/\/socialarchive.iath.virginia.edu\/control\/term#Person",
            "type": "entity_type"
        },
        "sources": [
            {
                "dataType": "Source",
                "language": {
                    "dataType": "Language",
                    "language": {
                        "id": "130",
                        "term": "eng",
                        "type": "language_code",
                        "description": "English"
                    },
                    "script": {
                        "id": "586",
                        "term": "Latn",
                        "type": "script_code",
                        "description": 

'Male'

In [13]:
text = response.text

type(text)

str

In [15]:
text_json = json.loads(text)

In [17]:
relations = text_json['constellation']['relations']

relations_data = pd.DataFrame([(i['targetConstellation'], i['content'], i['targetEntityType']['term']) for i in relations])
relations_data.columns = ['constellation_id', 'name', 'entity_type']
relations_data.head()

,constellation_id,name,entity_type
0,5532829,American Civil Liberties Union.,corporateBody
1,76111561,American Folklife Center,corporateBody
2,47287903,American Foundation For Equal Rights.,corporateBody
3,33162909,Antioch review.,corporateBody
4,47966548,Atlanta University Center (Ga.),corporateBody


In [23]:
relations[0]['targetEntityType']['term']

'corporateBody'

### People

In [19]:
people = relations_data[relations_data.entity_type == 'person'].reset_index()
people.shape

(155, 4)

In [23]:
people = people.drop('index', axis = 1)

In [25]:
people

,constellation_id,name,entity_type
0,1873330,"Avant, Clarence,",person
1,7797112,"Berry, Jason.",person
2,38219016,"Blackburn, Ruby Parks, 1901-1982.",person
3,17410868,"Bond, Horace Mann, 1904-1972.",person
4,33249158,"Bond, James, 1863-1929.",person
...,...,...,...
150,34868214,"Williams, Robert F. (Robert Franklin), 1925-1996.",person
151,68158725,"Winfrey, Oprah,",person
152,44738350,"Woodson, Carter Godwin, 1875-1950.",person
153,24120124,"Yarrow, Peter, 1938-",person


In [29]:
text_json['constellation'].keys()

dict_keys(['dataType', 'status', 'ark', 'entityType', 'sources', 'conventionDeclarations', 'languagesUsed', 'nameEntries', 'occupations', 'biogHists', 'relations', 'sameAsRelations', 'resourceRelations', 'places', 'subjects', 'nationalities', 'genders', 'id', 'version', 'dates'])

In [31]:
text_json['constellation']['biogHists'][0]['text']

"<biogHist><p>Civil rights activist, state representative, and state senator Julian Bond was born on January 14, 1940 in Nashville, Tennessee. He and his family moved to Pennsylvania, where his father, Horace Mann Bond, was appointed president of Lincoln University.</p><p>In 1957, Julian Bond graduated from the George School, a Quaker school in Bucks County, Pennsylvania, and entered Morehouse College. In 1960, Julian Bond was one of several hundred students who helped form the Student Nonviolent Coordinating Committee (SNCC). In 1965, Julian Bond was elected to the Georgia House of Representatives. He was barred from taking his seat in the House because of his outspoken statements against the Vietnam War. In December 1966 the Supreme Court ruled in his favor and he served four terms as Representative and six terms in the Georgia Senate, from 1975-1986. During the 1968 presidential election, he was the first African American to be nominated for Vice-President of the United States. He w

In [ ]:
# ['dates']['fromDate'], ['dates']['toDate']

In [33]:
content = text_json['constellation']
bond_info = (content['dates'][0]['fromDate'], content['dates'][0]['toDate'], content['sources'][0]['text'], content['nameEntries'][0]['original'])
bond_info

('1940-01-14',
 '2015-08-15',
 "Civil rights activist, state representative, and state senator Julian Bond was born on January 14, 1940 in Nashville, Tennessee. He and his family moved to Pennsylvania, where his father, Horace Mann Bond, was appointed president of Lincoln University.\r\n\r\nIn 1957, Julian Bond graduated from the George School, a Quaker school in Bucks County, Pennsylvania, and entered Morehouse College. In 1960, Julian Bond was one of several hundred students who helped form the Student Nonviolent Coordinating Committee (SNCC). In 1965, Julian Bond was elected to the Georgia House of Representatives. He was barred from taking his seat in the House because of his outspoken statements against the Vietnam War. In December 1966 the Supreme Court ruled in his favor and he served four terms as Representative and six terms in the Georgia Senate, from 1975-1986. During the 1968 presidential election, he was the first African American to be nominated for Vice-President of the 

In [115]:
%%time

people_data = []
attrs = [('dates', 'fromDate', 'birth'), ('dates', 'toDate', 'death'), ('biogHists', 'text', 'bio'), ('nameEntries', 'original', 'name')]

for idx, row in people.iterrows():
    data = {
            "command": "read",
            "constellationid": row['constellation_id']
        }
    query = json.dumps(data)
    response = requests.put(url, query)
    text_json = json.loads(response.text)
    content = text_json['constellation']
    info = {}
    if len(people_data) % 10 == 0:
        print(f'Retrieving record number {len(people_data)+1}')
    for attr in attrs:
        try:
            info[attr[2]] = content[attr[0]][0][attr[1]]
        except:
            info[attr[2]] = ''

    # gender (has different structure so needs to be separate)
    try:
        gender = content['genders'][0]['term']['term']
    except:
        gender = ''
    info['gender'] = gender
            
    # VIAF and ARKID
    
    viaf = ''

    try:
        for i in content['sameAsRelations']:
            if 'viaf' in i['uri']:
                viaf = i['uri']
    except:
        pass
    
    info['arkid'] = content['ark']
    info['viaf'] = viaf
    
    people_data.append(info)

Retrieving record number 1
Retrieving record number 11
Retrieving record number 21
Retrieving record number 31
Retrieving record number 41
Retrieving record number 51
Retrieving record number 61
Retrieving record number 71
Retrieving record number 81
Retrieving record number 91
Retrieving record number 101
Retrieving record number 111
Retrieving record number 121
Retrieving record number 131
Retrieving record number 141
Retrieving record number 151
CPU times: total: 6.2 s
Wall time: 11min 58s


Wall time lines up with the amount of time it took to run this script two years ago, which frankly makes sense because it's just the API that slows it down. If they allowed for more specific queries..... but no, the most granular I can get is a full read of the constellation. C'est la vie.

In [117]:
people_data = pd.DataFrame(people_data)

In [75]:
from bs4 import BeautifulSoup

In [121]:
people_data.head(10)

,birth,death,bio,name,gender,arkid,viaf
0,1943,2006,<biogHist><p>Music executive Clarence Avant wa...,"Avant, Clarence",,http://n2t.net/ark:/99166/w6nc7ws7,https://viaf.org/viaf/124757427
1,1966,1987,,"Berry, Jason (J. Jason)",,http://n2t.net/ark:/99166/w6s75tzr,https://viaf.org/viaf/78998014
2,1901,1982,"<biogHist><p xmlns=""urn:isbn:1-931666-33-4"">Ru...","Blackburn, Ruby Parks, 1901-1982.",,http://n2t.net/ark:/99166/w6c34h1f,
3,1904-11-08,1972-12-21,"<biogHist><p xmlns=""urn:isbn:1-931666-33-4"">Ed...","Bond, Horace Mann, 1904-1972",,http://n2t.net/ark:/99166/w63f4v8p,https://viaf.org/viaf/54950255
4,1863,1929,,"Bond, James, 1863-1929",,http://n2t.net/ark:/99166/w6zk5mz8,
5,1935,2009,,"Bond, Max, 1935-2009.",,http://n2t.net/ark:/99166/w6mk6j6v,
6,1902-10-13,1973-06-04,"<biogHist><p xmlns=""urn:isbn:1-931666-33-4"">Af...","Bontemps, Arna, 1902-1973",,http://n2t.net/ark:/99166/w6z329rw,https://viaf.org/viaf/17235442
7,1902-02-19,1992-12-27,"<biogHist><p xmlns=""urn:isbn:1-931666-33-4"">Ka...","Boyle, Kay, 1902-1992",,http://n2t.net/ark:/99166/w6q81d3s,https://viaf.org/viaf/49222285
8,1894,1985,"<biogHist><p xmlns=""urn:isbn:1-931666-33-4"">Ja...","Brawley, James P., 1894-1985,",,http://n2t.net/ark:/99166/w6pg4tvh,
9,1938-08-15,,"<biogHist><p xmlns=""urn:isbn:1-931666-33-4"">St...","Breyer, Stephen G., 1938-....",,http://n2t.net/ark:/99166/w6vj6cbd,https://viaf.org/viaf/108725142


In [123]:
def clean_bio(text):
    replace_map = {
        '</p>' : ' ',
        '<span>' : ' ',
        '</span>' : ' '
    }

    if text == '':
        return text

    for k, v in replace_map.items():
        text = text.replace(k, v)

    return BeautifulSoup(text).get_text()

In [125]:
people_data.bio = people_data.bio.apply(clean_bio)
people_data

,birth,death,bio,name,gender,arkid,viaf
0,1943,2006,Music executive Clarence Avant was born on Feb...,"Avant, Clarence",,http://n2t.net/ark:/99166/w6nc7ws7,https://viaf.org/viaf/124757427
1,1966,1987,,"Berry, Jason (J. Jason)",,http://n2t.net/ark:/99166/w6s75tzr,https://viaf.org/viaf/78998014
2,1901,1982,"Ruby Georgia Parks Blackburn (1901-1982), beau...","Blackburn, Ruby Parks, 1901-1982.",,http://n2t.net/ark:/99166/w6c34h1f,
3,1904-11-08,1972-12-21,"Educator, sociologist, scholar, and author. \n...","Bond, Horace Mann, 1904-1972",,http://n2t.net/ark:/99166/w63f4v8p,https://viaf.org/viaf/54950255
4,1863,1929,,"Bond, James, 1863-1929",,http://n2t.net/ark:/99166/w6zk5mz8,
...,...,...,...,...,...,...,...
150,1925-02-26,1996-10-15,"Black poet and militant civil rights activist,...","Williams, Robert F. (Robert Franklin), 1925-1996",,http://n2t.net/ark:/99166/w6xg9zv7,https://viaf.org/viaf/23915862
151,1954-01-29,,"Oprah Winfrey is an American talk show host, t...","Winfrey, Oprah, 1954-",,http://n2t.net/ark:/99166/w6d79cm0,https://viaf.org/viaf/27226716
152,1875-12-19,1950-04-03,"Carter Godwin Woodson, educator and historian,...","Woodson, Carter Godwin, 1875-1950",,http://n2t.net/ark:/99166/w6h23s6h,https://viaf.org/viaf/19770339
153,1938-05-31,,Folk singers. \n From the descripti...,"Yarrow, Peter, 1938-....",,http://n2t.net/ark:/99166/w6b56qxq,https://viaf.org/viaf/84167935


In [127]:
people_data.gender.value_counts()

gender
          115
Male       35
Female      5
Name: count, dtype: int64

In [136]:
people_data.viaf.value_counts()['']

np.int64(25)

In [138]:
people_data.bio.value_counts()['']

np.int64(48)

In [148]:
people_data.death.value_counts()['']

np.int64(57)

So, 8 missing birth dates, 57 missing death dates (presumably many being people who have not yet died), 48 with no bio and 25 with no corresponding VIAF link. Oh an dmost of them are missing gender info on SNAC, but I don't think that's a huge deal. All in all, seems successful. Going to export with everything included for now, but may need to trim things down. Not sure how useful the gender column will be. Happy with the output though.

In [153]:
people_data.to_csv('SNAC_people.csv')

### Places, Corporations, etc.

In [163]:
relations_data.entity_type.value_counts()

entity_type
person           155
corporateBody     53
family             3
Name: count, dtype: int64

In [165]:
# going to come back around to this TBH. Doesn't seem like it adds as much as the "People" stuff